In [2]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
def Weight_value(shape):
    init = tf.random_normal(shape, stddev=0.1)
    return tf.Variable(init, name="weight")
def bias_value(shape):
    init = tf.constant(0.1, shape=shape)
    return tf.Variable(init)
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")
def pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME")

X = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_image = tf.reshape(X, [-1, 28, 28, 1])

# layer1 conv1  [-1, 28, 28, 32]
W_conv1 = Weight_value([5, 5, 1, 32])
b_conv1 = bias_value([32])
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1)+b_conv1)
# layer2 pool1 [-1, 14, 14, 32]
h_pool1 = pool_2x2(h_conv1)
# layer3 conv2 [-1, 14, 14, 64]
W_conv2 = Weight_value([5, 5, 32, 64])
b_conv2 = bias_value([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2)+b_conv2)
# layer4 pool2 [-1,7,7,64]
h_pool2 = pool_2x2(h_conv2)
# layer5 fc1 [-1,1024]
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
W_fc1 = Weight_value([7*7*64, 1024])
b_fc1 = bias_value([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1)+b_fc1)
#layer6 dropout
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
# layer7 fc2 [-1,10]
W_fc2 = Weight_value([1024, 10])
b_fc2 = bias_value([10])
y_conv = tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2)+b_fc2)

# cross_entropy
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
# optimizer
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
# accuracy
correct_prediction = tf.equal(tf.argmax(y_, 1), tf.argmax(y_conv, 1))
acc = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# init
init = tf.global_variables_initializer()
# sess
test_holder = {X: mnist.test.images, y_: mnist.test.labels, keep_prob: 1.0}
display_step = 100
test_acc_list = []
with tf.Session() as sess:
    sess.run(init)
    for i in range(20000):
        batch_xs, batch_ys = mnist.train.next_batch(50)
        sess.run(train_step, feed_dict={X: batch_xs, y_: batch_ys, keep_prob: 0.75})
        if i % display_step == 0:
            test_acc = acc.eval(feed_dict=test_holder)
            test_acc_list.append(test_acc)
            print("epoch %d, test_acc: %.4f" % (i, test_acc))

ResourceExhaustedError: OOM when allocating tensor with shape[10000,32,28,28]
	 [[Node: Conv2D_4 = Conv2D[T=DT_FLOAT, data_format="NHWC", padding="SAME", strides=[1, 1, 1, 1], use_cudnn_on_gpu=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Reshape_4, weight_8/read)]]
	 [[Node: Mean_5/_33 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_76_Mean_5", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Conv2D_4', defined at:
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Anaconda3\li